In [92]:
from agents import Agent, handoff, RunContextWrapper, Runner, AsyncOpenAI, set_default_openai_client, set_default_openai_api
from agents.tool import function_tool
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from dotenv import load_dotenv
import os
import asyncio
import gradio as gr


In [93]:
load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_MODEL = os.getenv("AZURE_OPENAI_MODEL")

OPEN_WEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")


external_client = AsyncOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    base_url=f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{AZURE_OPENAI_DEPLOYMENT_NAME}",
    default_headers={"api-key": AZURE_OPENAI_API_KEY},
    default_query={"api-version": AZURE_OPENAI_API_VERSION}
)

set_default_openai_client(external_client, use_for_tracing=False)
set_default_openai_api("chat_completions")

In [94]:
from axios import Axios

httpClient = Axios()

In [95]:
class GeoLocation:
  def __init__(self, name, lat, lon):
    self.name = name
    self.lat = lat
    self.lon = lon

  @classmethod
  def from_json(cls, json_data: dict):
    return cls(
      name=json_data.get('name'),
      lat=json_data.get('lat'),
      lon=json_data.get('lon')
    )
  
  def to_dict(self):
    return {
      "name": self.name,
      "lat": self.lat,
      "lon": self.lon
    }

  def __repr__(self):
    return f"GeoLocation(name='{self.name}', lat='{self.lat}', lon='{self.lon}')"
  

@function_tool
async def get_geo(city: str):
  """
  Get geolocation details of a city.
  Parameters:
    - city (str): The name of the city to get location info for.
  Returns:
    - {"lat": float, "lon": float}: Basic location info or coordinates.
  """
  url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&appid={OPEN_WEATHER_API_KEY}"
  response = await httpClient.aget(url)
  geo = GeoLocation.from_json(response.data[0])
  print(geo)
  return geo.to_dict()


In [96]:
class WeatherInfo():
  def __init__(self, main, temp_min, temp_max):
    self.main = main
    self.temp_min = temp_min
    self.temp_max = temp_max

  @classmethod
  def from_json(cls, json_data: dict):
    weather = json_data.get('weather')
    main = json_data.get('main')

    return cls(
      main=weather[0].get('main'),
      temp_min=main.get('temp_min'),
      temp_max=main.get('temp_max')
    )
  
  def to_dict(self):
    return {
      "main": self.main,
      "temp_min": self.temp_min,
      "temp_max": self.temp_max
    }
  
  def __repr__(self):
    return f"WeatherInfo(main='{self.main}', min='{self.temp_min}', max='{self.temp_max}')"

 
@function_tool
async def get_forecast(lat: float, lon: float):
  """
  Get weather details of a city.
  Parameters:
    - lat (float): Latitude of a city.
    - lon (float): Longitude of a city.
  Returns:
    - {"main": string, "temp_min": number, "temp_max": number}: Weather info contains main, min temperature and max temperature.
  """
  url=f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OPEN_WEATHER_API_KEY}&units=metric"
  response = await httpClient.aget(url)
  info = WeatherInfo.from_json(response.data)
  print(info)
  return info.to_dict()  

In [97]:
geo_instruction = f"""
{RECOMMENDED_PROMPT_PREFIX} \
You extract the city name from the text in any given language. \
You only use the given tools. \
You get the result using get_geo tool using the city name. \
Transfer back the lat and lon to Forecast agent once done.
"""

geo_agent = Agent(
  name = "Geo Agent",
  instructions = geo_instruction,
  tools = [get_geo]
)

weather_instruction = f"""
{RECOMMENDED_PROMPT_PREFIX} \
You find the weather info using a given lat and lon. \
You only use the given tools. \
Given the lat and lon you find the weather info using get_forecast \
Transfer back the result to Forecast agent once done.
"""

weather_agent = Agent(
  name = "Weather Agent",
  instructions = weather_instruction,
  tools = [get_forecast]
)


instructions = f"""
{RECOMMENDED_PROMPT_PREFIX}
You are a weather forecast agent. You try to understand the user's prompt in any language.
You are not allowed to solve the task yourself. You only delegate (handoff) tasks to Geo Agent or Weather Agent.

Your workflow is as follows:
1. Extract or ask for the city name if it's not already provided.
2. Handoff the city name to the Geo Agent to get the lat and lon.
3. Handoff the lat and lon to the Weather Agent to get the weather.
4. Display the weather info


NEVER stop at step 2. You MUST alwasy proceed to step 3 once the lat and lon are available.
ALWAYS pass the lat and lon to weather_agent as input when the lat and lon are available.
ALWAYS reply in Mandarin.
"""


triage_agent = Agent(
  name = "Forecast Agent",
  instructions = instructions,
  handoffs= [geo_agent, weather_agent]
)

geo_agent.handoffs.append(triage_agent)
weather_agent.handoffs.append(triage_agent)

messages = []
async def chat_weather(prompt, history):
  result = await Runner.run(triage_agent, prompt)
  return result.final_output

In [ ]:
def greet(message, history, audio):
  print(history)
  return "Hello, " + message +"!"


microphone = gr.Microphone()

gr.ChatInterface(
  fn=chat_weather, 
  type="messages"
).launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


GeoLocation(name='Cheras', lat='3.0991919', lon='101.7374229')
WeatherInfo(main='Rain', min='30.62', max='30.62')
GeoLocation(name='Ipoh', lat='4.5986817', lon='101.0900236')
WeatherInfo(main='Clouds', min='27.86', max='29.93')
